In [3]:
import numpy as np
import pandas as pd
import glob
import time
import pandas as pd
import os
import matplotlib.pyplot as plt
import keras
import cv2
import warnings
# Suppress TensorFlow deprecation warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="tensorflow")


In [4]:
df = pd.read_csv('XSS_dataset.csv')

In [5]:
df.head()

,Unnamed: 0,Sentence,Label
0,0,"<li><a href=""/wiki/File:Socrates.png"" class=""i...",0
1,1,"<tt onmouseover=""alert(1)"">test</tt>",1
2,2,"\t </span> <span class=""reference-text"">Steeri...",0
3,3,"\t </span> <span class=""reference-text""><cite ...",0
4,4,"\t </span>. <a href=""/wiki/Digital_object_iden...",0


In [6]:

df=df[df.columns[-2:]]  # Only get sentence and labels
df.head()


,Sentence,Label
0,"<li><a href=""/wiki/File:Socrates.png"" class=""i...",0
1,"<tt onmouseover=""alert(1)"">test</tt>",1
2,"\t </span> <span class=""reference-text"">Steeri...",0
3,"\t </span> <span class=""reference-text""><cite ...",0
4,"\t </span>. <a href=""/wiki/Digital_object_iden...",0


In [7]:

# Get Sentences data from data frame
sentences=df['Sentence'].values
sentences[1]

print(len(sentences))

# Convert to ASCII


13686


In [8]:
def convert_to_ascii(sentence):
    sentence_ascii=[]

    for i in sentence:
        """Some characters have values very big e.d 8221 adn some are chinese letters
        I am removing letters having values greater than 8222 and for rest greater
        than 128 and smaller than 8222 assigning them values so they can easily be normalized"""
        if(ord(i)<8222):      # ” has ASCII of 8221
            if(ord(i)==8217): # ’  :  8217
                sentence_ascii.append(134)
            if(ord(i)==8221): # ”  :  8221
                sentence_ascii.append(129)
            if(ord(i)==8220): # “  :  8220
                sentence_ascii.append(130)
            if(ord(i)==8216): # ‘  :  8216
                sentence_ascii.append(131)
            if(ord(i)==8217): # ’  :  8217
                sentence_ascii.append(132)
            if(ord(i)==8211): # –  :  8211
                sentence_ascii.append(133)

            """
            If values less than 128 store them else discard them
            """
            if (ord(i)<=128):
                    sentence_ascii.append(ord(i))
            else:
                    pass

    zer=np.zeros((10000))
    for i in range(len(sentence_ascii)):
        zer[i]=sentence_ascii[i]

    zer.shape=(100, 100)

    return zer


In [9]:

# send each sentence to be converted to ASCII

arr=np.zeros((len(sentences),100,100))

for i in range(len(sentences)):
    image=convert_to_ascii(sentences[i])
    x=np.asarray(image,dtype='float')
    image =  cv2.resize(x, dsize=(100,100), interpolation=cv2.INTER_CUBIC)
    image/=128
    arr[i]=image

print("Input data shape : ", arr.shape)


Input data shape :  (13686, 100, 100)


In [10]:

# Reshape data for input to CNN
data = arr.reshape(arr.shape[0], 100, 100, 1)

data.shape


(13686, 100, 100, 1)

In [12]:

y = df['Label'].values
print(y)

[0 1 0 ... 1 0 0]


In [13]:

# Split into train/test data
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(data, y, test_size=0.2, random_state=42)


In [14]:

# import libraries for Making Model
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout, MaxPool2D, BatchNormalization

# A basic CNN Model
# Number of layers = 11
# Number of Convolutional layer: 3

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation=tf.nn.relu, input_shape=(100, 100, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()






Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2  (None, 49, 49, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 23, 23, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 21, 256)       295168    
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 10, 10, 256)     

In [15]:
# Stop when validation accuracy > 97

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy') > 0.97):
            print(" \n Reached 97% + validation accuracy")
            self.model.stop_training = True

callbacks = myCallback()

batch_size = 128
num_epoch = 10
#model training
model_log = model.fit(trainX, trainY,
                      batch_size=batch_size,
                      epochs=num_epoch,
                      verbose=1,
                      validation_data=(testX, testY)
#                      callbacks=[callbacks]
                     )


Epoch 1/10


86/86 [==============================] - 137s 1s/step - loss: 0.4448 - accuracy: 0.7924 - val_loss: 0.3801 - val_accuracy: 0.8181
Epoch 2/10
86/86 [==============================] - 121s 1s/step - loss: 0.2339 - accuracy: 0.9024 - val_loss: 0.0854 - val_accuracy: 0.9744
Epoch 3/10
86/86 [==============================] - 126s 1s/step - loss: 0.0766 - accuracy: 0.9776 - val_loss: 0.0776 - val_accuracy: 0.9766
Epoch 4/10
86/86 [==============================] - 135s 2s/step - loss: 0.0577 - accuracy: 0.9839 - val_loss: 0.0530 - val_accuracy: 0.9847
Epoch 5/10
86/86 [==============================] - 132s 2s/step - loss: 0.0495 - accuracy: 0.9864 - val_loss: 0.0586 - val_accuracy: 0.9854
Epoch 6/10
86/86 [==============================] - 131s 2s/step - loss: 0.0426 - accuracy: 0.9880 - val_loss: 0.0460 - val_accuracy: 0.9858
Epoch 7/10
86/86 [==============================] - 130s 2s/step - loss: 0.0375 - accuracy: 0.9900 - val_loss: 0.0488 - val_accuracy: 0.9828
Epoch 8/10


In [16]:

# predict for test set
pred = model.predict(testX)


86/86 [==============================] - 8s 83ms/step


In [17]:

# Threshold values predicted

for i in range(len(pred)):
    if pred[i] > 0.5:
        pred[i] = 1
    elif pred[i] <= 0.5:
        pred[i] = 0


In [18]:

# Count True predicted and wrong predicted

true = 0
false = 0

for i in range(len(pred)):
    if pred[i] == testY[i]:
        true += 1
    else:
        false += 1

print("correct predicted :: ", true)
print("false prediction :: ", false)



correct predicted ::  2710
false prediction ::  28


In [19]:
# Number of attack and benign data in test set

attack = 0
benign = 0
for i in range(len(testY)):
    if testY[i] == 1:
        attack += 1
    else:
        benign += 1

print("Attack data in test set :: ", attack)
print(" Benign data in test set :: ", benign)


Attack data in test set ::  1478
 Benign data in test set ::  1260


In [20]:


"""### Confusion Matrix"""

# Takes True positive, true negative, false positive and false negative as input
# Returns Accuracy

def accuracy_function(tp, tn, fp, fn):
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    return accuracy

# Takes True Positive and false positive
# Returns Precision

def precision_function(tp, fp):
    precision = tp / (tp + fp)
    return precision

# Takes True Positive and false negative
# Returns Recall

def recall_function(tp, fn):
    recall = tp / (tp + fn)
    return recall

# Takes true values and predicted


In [21]:
# Returns Accuracy, precision and recall

def confusion_matrix(truth, predicted):
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0

    for true, pred in zip(truth, predicted):
        if true == 1:
            if pred == true:
                true_positive += 1
            elif pred != true:
                false_negative += 1
        elif true == 0:
            if pred == true:
                true_negative += 1
            elif pred != true:
                false_positive += 1

    accuracy = accuracy_function(true_positive, true_negative, false_positive, false_negative)
    precision = precision_function(true_positive, false_positive)
    recall = recall_function(true_positive, false_negative)

    return accuracy, precision, recall

accuracy, precision, recall = confusion_matrix(testY, pred)

print(" Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))


 Accuracy : 0.9897735573411249 
 Precision : 0.9852744310575636 
 Recall : 0.9959404600811907


In [25]:
def preprocess_new_data(sentence):
    image = convert_to_ascii(sentence)
    x = np.asarray(image, dtype='float')
    image = cv2.resize(x, dsize=(100, 100), interpolation=cv2.INTER_CUBIC)
    image /= 128
    return image.reshape(1, 100, 100, 1)

# Example new data for prediction
new_sentence = '<caption onpointerdown=alert(1)>XSS</caption>'
new_data = preprocess_new_data(new_sentence)

# Make predictions on new data
new_predictions = model.predict(new_data)

# Apply a threshold to convert probabilities to binary predictions
binary_prediction = (new_predictions > 0.5).astype(int)

# Display the result
if binary_prediction[0][0] == 1:
    print("Prediction: XSS Attack")
else:
    print("Prediction: Benign")


1/1 [==============================] - 0s 218ms/step
Prediction: XSS Attack
